In [4]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates

from catboost import CatBoostRegressor
import datetime
from datetime import timedelta, date

In [13]:
def prepro(X):
    ext_df = pd.read_csv('external_data/external_data.csv')
    ext_df['date'] = pd.to_datetime(ext_df['date'].values.astype('<M8[us]'))
    data = utils.is_jour_ferie(
    utils.is_lockdown(
        utils.create_x_weather(
            utils.apply_couvre_feu(
                utils.filter_columns(X)
            )
        )
    )
    )
    result = utils._encode_dates(data).drop(columns=['date'])
    return result


In [27]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [50]:
X, y = utils.get_train_data()
#X = X[X['date']>='2021-04-1'].sort_values(by=['counter_id', 'date']).reset_index().drop(columns='index')
X = prepro(X)
y = pd.Series(y)[:X.shape[0]]


num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']

preprocessing = FunctionTransformer(prepro)

col_transformer = ColumnTransformer(
    transformers=[
        
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features)
    ],
    remainder='passthrough'
)


X_scaled = col_transformer.fit_transform(X)

# Création des séquences
sequence_length = 24  # 24 heures de données par séquence
num_sequences_per_counter = 3888 // sequence_length

sequences = []
targets = []

for counter_id in X['counter_id'].unique():
    counter_index = X[X['counter_id'] == counter_id].index
    counter_data = X_scaled[counter_index]
    counter_target = y[counter_index]

    for i in range(num_sequences_per_counter):
        start_index = i * sequence_length
        end_index = start_index + sequence_length

        if end_index <= counter_data.shape[0]:
            sequence = counter_data[start_index:end_index]
            target = counter_target.iloc[end_index - 1]

            sequences.append(sequence)
            targets.append(target)

X_sequences = np.array(sequences)
y_targets = np.array(targets)



IndexError: index 496845 is out of bounds for axis 0 with size 496827

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_targets, test_size=0.2, random_state=42)

# Construction du modèle LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
#model.add(Dropout(0.2))  # Ajout d'une couche Dropout pour la régularisation

# Deuxième couche LSTM
model.add(LSTM(50, activation='relu', return_sequences=True))
#model.add(Dropout(0.2))  # Ajout d'une couche Dropout pour la régularisation

# Troisième couche LSTM#model.add(Dropout(0.2))  # Ajout d'une couche Dropout pour la régularisation

# Couche Dense
model.add(Dense(50, activation='relu'))
#model.add(Dropout(0.2))  # Ajout d'une couche Dropout pour la régularisation

model.add(Dense(1))  # Ajustez en fonction de votre tâche (régression ou classification)

# Compilation du modèle
model.compile(optimizer='Adagrad', loss='mean_squared_error')  # Utilisez 'categorical_crossentropy' pour la classification

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Évaluation du modèle
mse = model.evaluate(X_test, y_test)
print(f"Mean Squared Error: {mse}")

Epoch 1/100
227/227 [==============================] - 3s 9ms/step - loss: 9013.7188 - val_loss: 1800.2255
Epoch 2/100
227/227 [==============================] - 2s 9ms/step - loss: 1005.9053 - val_loss: 574.7899
Epoch 3/100
 68/227 [=======>......................] - ETA: 1s - loss: 517.6368

KeyboardInterrupt: 